## Installing Pytorch

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58076000 @  0x7f03e4acb2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


## Building Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 2998 (delta 12), reused 7 (delta 7), pack-reused 2973
Receiving objects: 100% (2998/2998), 3.06 MiB | 5.26 MiB/s, done.
Resolving deltas: 100% (2185/2185), done.


In [0]:
import os
os.chdir("fairseq/")

In [4]:
!pip install -r requirements.txt
%run -i 'setup.py' build develop

    100% |████████████████████████████████| 430kB 23.7MB/s 
    100% |████████████████████████████████| 163kB 29.6MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/scripts
copying scripts/__init__.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/average_checkpoints.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/read_binarized.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/build_sym_alignment.py -> build/lib.linux-x86_64-3.6/scripts
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/options.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_generator.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/multiprocessing_pdb.py -> build/lib.linux-x86_64-3.6/fairseq
co

## Model for Afrikaans
### Data Preprop
Upload the data in the form train.en, train.tn, valid.en, valid.tn, test.en, test.tn.

### Subword

In [5]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 481, done.
remote: Total 481 (delta 0), reused 0 (delta 0), pack-reused 481
Receiving objects: 100% (481/481), 203.75 KiB | 742.00 KiB/s, done.
Resolving deltas: 100% (287/287), done.


In [0]:
os.chdir('../')

In [7]:
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

# Learn a vocabulary using 40,000 merge operations
!fairseq/subword-nmt/learn_bpe.py -s 4000 <combine.txt> enaf.codes

# Apply the vocabulary to the training file
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='enaf.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

In [0]:
os.chdir('fairseq/')

In [9]:
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

Namespace(alignfile=None, destdir='data-bin/enaf', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='en', srcdict=None, target_lang='af', testpref='../test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='../train', validpref='../valid', workers=1)
| [en] Dictionary: 3511 types
| [en] ../train.en: 37219 sents, 790613 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 3511 types
| [en] ../valid.en: 12953 sents, 383339 tokens, 0.114% replaced by <unk>
| [en] Dictionary: 3511 types
| [en] ../test.en: 3000 sents, 76516 tokens, 0.0562% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../train.af: 37219 sents, 813055 tokens, 0.0% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../valid.af: 12953 sents, 403141 tokens, 0.344% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../test.af: 3000 sents, 78473 tokens, 0.0624% replaced by <unk>
| Wrote preprocessed data to data-bin/enaf


### Train Model

In [10]:
!mkdir -p chckpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir ckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

Namespace(arch='fconv', bucket_cap_mb=150, clip_norm=0.1, criterion='cross_entropy', data=['data-bin/enaf'], ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=512, decoder_embed_path=None, decoder_layers='[(512, 3)] * 20', decoder_out_embed_dim=256, device_id=0, distributed_backend='nccl', distributed_init_method=None, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, encoder_embed_dim=512, encoder_embed_path=None, encoder_layers='[(512, 3)] * 20', fix_batches_to_gpus=False, fp16=False, fp16_init_scale=128, fp16_scale_window=None, keep_interval_updates=-1, left_pad_source='True', left_pad_target='False', log_format=None, log_interval=1000, lr=[0.25], lr_scheduler='reduce_lr_on_plateau', lr_shrink=0.1, max_epoch=0, max_sentences=None, max_sentences_valid=None, max_source_positions=1024, max_target_positions=1024, max_tokens=4000, max_update=0, min_loss_scale=0.0001, min_lr=1e-05, momentum=0.99, no_epoch_checkpoints=False, no_progress_bar=False

### Testing
Generate translations from the test data. Calculate the Bleu score.

In [0]:
# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**8k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 8000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**12k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 12000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir ceckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ceckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**16k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 16000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chekpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path chekpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**20k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 20000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path chckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**24k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 24000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**28k**

## Results

*   40k: 
*   36k:
*   32k: 
*   28k:
*   24k: 
*   20k: 
*   16k: 
*   12k: 
*     8k: 
*     4k: 25.04

